# Janus gym environment

In [332]:
import gym
from gym import spaces
import random
import stable_baselines3
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import DQN, DDPG
from math import sqrt
import numpy as np
import pandas as pd
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import preprocessing
from sklearn.datasets import make_regression

import math
import pickle

class Janus(gym.Env):
    metadata = {'render.modes': ['human']}
    template_filename = 'data/dataset-S_public/public/dataset_S-{}.csv'

    def __init__(self):
        super(Janus, self).__init__()
        #actions: move on the grid, by continuous value in -1,1
        #0,0 no move
        #based on 94 controlable parameters
        #"We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) ", we will multiply effect by 2
#         self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(94, ))
        #we focus on the 1 most influencal action
        nbr_actions = 4
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(nbr_actions, ))
    

        # all the observation_space
        self.observation_space = spaces.Box(low=-1.0, high=1.0, shape=(228, ))

        file1 = pd.read_csv(self.template_filename.format('file1'), index_col=0)
        file2 = pd.read_csv(self.template_filename.format('file2'), index_col=0)
        file3 = pd.read_csv(self.template_filename.format('file3'), index_col=0)
        vav = pd.read_csv(self.template_filename.format('VAV'), index_col=0)
        self.ti = pd.read_csv(self.template_filename.format('TI'), index_col=0)
        self.ts = pd.read_csv(self.template_filename.format('TS'), index_col=0)
        x_df = file1.copy()
        x_df = x_df.loc[:, (x_df != 0).any(axis=0)]  ## remove static columns
        x_df = x_df.fillna(x_df.mean())  ## replace nan with mean

        self.y_df = file2.copy()
        self.y_df.dropna(how='all', axis=1,
                    inplace=True)  ## drop full tempty columns
        self.y_df = self.y_df.fillna(self.y_df.mean())

        self.vav_df = vav.copy()
        # Dropping few columns
        for dataset in [self.y_df, self.vav_df, self.ti, self.ts]:
            dataset.drop(['target_1', 'target_2', 'target_3', 'target_4'],
                  axis=1,
                  inplace=True)  #to simplify with a 2-dimension target space

        print('features shape: {}, \ntargets shape: {}'.format(
            x_df.shape, self.y_df.shape))

        x_train, x_test, y_train, y_test = train_test_split(x_df,
                                                            self.y_df,
                                                            test_size=0.1,
                                                            random_state=14)
        print('\nLength of train is {}, test is {}'.format(
            len(x_train), len(x_test)))
        ## Random forest
        filename = 'data/models/janus_RF.pkl'  # janus_LinearReg, janus_RF

        # pickle.dump(ml_model, open(filename, 'wb'))

        # # load the model from disk
        self.ml_model = pickle.load(open(filename, 'rb'))
        print(f'R squared: {self.ml_model.score(x_test, y_test.values):0.04f}')

        self.full_x = file3.copy()[x_df.columns]
        self.full_x = self.full_x.fillna(x_df.mean())
        
        self.partial_x = x_train.copy()

        inferred_y = pd.DataFrame(self.ml_model.predict(self.full_x),
                                  columns=self.y_df.columns)

        # list of [min, max, step, range] for each var
        scale = 100
        decimals = 3
        
        self.list_important_actions = np.argsort(self.ml_model.feature_importances_[:94])[::-1][:nbr_actions]

        ## get limits for Rewards
        self.output_steps = [round((self.y_df[i].max() - self.y_df[i].min())/scale, decimals) \
                        for i in self.y_df.columns]
        print('Output steps: ', self.output_steps)

    def reset(self):
#         self.current_position = self.revert_to_obs_space(
#             self.full_x.sample().values.reshape(-1), self.full_x)
        random.seed(13)
        idx = random.randint(0,len(janus_env.partial_x)-1)
        idx = 47
        self.current_position = self.revert_to_obs_space(
            self.full_x.iloc[idx].values.reshape(-1), self.full_x)
        
        self.last_action = np.array([])
        self.last_effect = False
        self.global_reward = 0
        self.episode_length = 0
        #print(f'reset at position {self.current_position[:10]}...')
        return self.current_position

    def step(self, action):
#         self.current_position[0:len(action)] = action
        for index, act in enumerate(self.list_important_actions):
            self.current_position[act]=action[index]
        self.last_action = action
        self.episode_length += 1
        
        reward = self.discrete_reward_from_obs(
            self.convert_to_real_obs(self.current_position,
                                     self.full_x).values.reshape(1,-1))
        done = reward >= -0.1*self.y_df.shape[1]
        if done:
            reward = 100
        
        if self.episode_length>100:
            #print('episode too long -> reset')
            done = True
            
        if (max(abs(action))):
            # if on border, we kill episode
            done = True
            

        self.global_reward += reward
        return self.current_position, reward, done, {}

    def render(self):
        print(
            f'position {self.current_position[:10]}, action {self.last_action[:5]}, effect {self.last_effect}, done {done}, global_reward {self.global_reward}'
        )

    def convert_to_real_obs(self, observation, observation_dataset):
        '''
        to convert an observation from observation space ([-1, 1],325) to  real world
        -1 matches with min() of each column
        1 matches with max() of each column
        
        observation: instance of observation_space
        observation_dataset: the full real dataset (obfuscated in that case)
        '''
        return (observation + np.ones(self.observation_space.shape)) / 2 * (
            observation_dataset.max() -
            observation_dataset.min()) + observation_dataset.min()

    def revert_to_obs_space(self, real_observation, observation_dataset):
        '''
        to revert an observation sample (from real world) to observation space
        min() of each column will match with -1
        max() of each column will match with +1
        
        real_observation: instance of real_world
        observation_dataset: the full real dataset (obfuscated in that case)
        '''
        return np.nan_to_num(
            2 * (real_observation - observation_dataset.min()) /
            (observation_dataset.max() - observation_dataset.min()) -
            np.ones(self.observation_space.shape)).reshape(-1)

    def discrete_reward_from_obs(self, observation):
        ''' Discrete reward 
        observation if from real world not observation space
        '''

        new_y = self.ml_model.predict(observation).reshape(-1)
        return self.discrete_reward_continuous(new_y)

    def discrete_reward(self, new_y):
        ''' Discrete reward '''

        new_val = [
            sqrt((self.vav_df.iloc[:, i].values[0] - new_y[i])**2)
            for i in range(len(new_y))
        ]
        k = 10
        k1 = 1
        if new_val[0] < k * self.output_steps[0] and new_val[
                1] < k * self.output_steps[1]:
            reward = 1  #dans les 10% d'amplitude max autour de la vav
            if new_val[0] < k1 * self.output_steps[0] and new_val[
                    1] < k1 * self.output_steps[1]:
                reward = 10  #dans les 1% d'amplitude max autour de la vav
                on_target = True
#                 print('On Target : ', new_y)
        else:
            reward = -1
        return reward
    
    
    
    def discrete_reward_continuous(self, new_y):
        ''' Continuous reward '''
        final_reward = 0 

        for i in range(len(new_y)):
            reward = -9
            if ( self.ti.iloc[:,i].values[0] <=  new_y[i] <= self.ts.iloc[:,i].values[0]):
                if ( new_y[i] >= self.vav_df.iloc[:,i].values[0] ):
                    reward = 1-(new_y[i]-self.vav_df.iloc[:,i].values[0])/(self.ts.iloc[:,i].values[0]-self.vav_df.iloc[:,i].values[0])
                else:
                    reward = 1-(new_y[i]-self.ti.iloc[:,i].values[0])/(self.vav_df.iloc[:,i].values[0]-self.ti.iloc[:,i].values[0])
            reward += -1
            final_reward+=reward
    #         print(f'reward {reward} final_reward {final_reward} i {i}')

        if (final_reward>0.7*len(new_y)):
            on_target = True
    #         print('On Target : ', new_y)

        return final_reward




In [317]:
janus_env = Janus()
from stable_baselines3.common.env_checker import check_env

check_env(janus_env)

features shape: (782, 228), 
targets shape: (782, 2)

Length of train is 703, test is 79
R squared: 0.6550
Output steps:  [0.064, 0.057]


AssertionError: The `done` signal must be a boolean

# SAC training

In [127]:
from stable_baselines3 import SAC

janus_env = Janus()
check_env(janus_env)

model_janus_sac = SAC("MlpPolicy", janus_env, verbose=2,tensorboard_log="./tensorboard/")
model_janus_sac.learn(total_timesteps=100000, log_interval=4, tb_log_name="janus partial sac")

janus_env.reset()
for i in range(100):
    action, _ = model_janus_sac.predict(janus_env.current_position)
    print(f'action {action}')
    obs, rewards, done, info = janus_env.step(action)
    janus_env.render()
    if done: break
janus_env.close()

features shape: (782, 228), 
targets shape: (782, 2)

Length of train is 703, test is 79
R squared: 0.6550
Output steps:  [0.064, 0.057]
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./tensorboard/janus partial sac_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 27       |
|    time_elapsed    | 14       |
|    total timesteps | 404      |
| train/             |          |
|    actor_loss      | 0.392    |
|    critic_loss     | 0.0128   |
|    ent_coef        | 0.913    |
|    ent_coef_loss   | -0.153   |
|    learning_rate   | 0.0003   |
|    n_updates       | 303      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/   

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 60       |
|    fps             | 24       |
|    time_elapsed    | 244      |
|    total timesteps | 6060     |
| train/             |          |
|    actor_loss      | 16.6     |
|    critic_loss     | 0.957    |
|    ent_coef        | 0.168    |
|    ent_coef_loss   | -3.01    |
|    learning_rate   | 0.0003   |
|    n_updates       | 5959     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 64       |
|    fps             | 24       |
|    time_elapsed    | 260      |
|    total timesteps | 6464     |
| train/             |          |
|    actor_loss      | 17.7     |
|    critic_loss     | 3.35     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 120      |
|    fps             | 23       |
|    time_elapsed    | 517      |
|    total timesteps | 12120    |
| train/             |          |
|    actor_loss      | 30.4     |
|    critic_loss     | 17.2     |
|    ent_coef        | 0.0301   |
|    ent_coef_loss   | -2.55    |
|    learning_rate   | 0.0003   |
|    n_updates       | 12019    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 124      |
|    fps             | 23       |
|    time_elapsed    | 536      |
|    total timesteps | 12524    |
| train/             |          |
|    actor_loss      | 31       |
|    critic_loss     | 3.8      |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 180      |
|    fps             | 22       |
|    time_elapsed    | 799      |
|    total timesteps | 18180    |
| train/             |          |
|    actor_loss      | 39.1     |
|    critic_loss     | 46.2     |
|    ent_coef        | 0.00898  |
|    ent_coef_loss   | -0.818   |
|    learning_rate   | 0.0003   |
|    n_updates       | 18079    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 184      |
|    fps             | 22       |
|    time_elapsed    | 817      |
|    total timesteps | 18584    |
| train/             |          |
|    actor_loss      | 39.4     |
|    critic_loss     | 17.5     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 240      |
|    fps             | 22       |
|    time_elapsed    | 1059     |
|    total timesteps | 24240    |
| train/             |          |
|    actor_loss      | 42.9     |
|    critic_loss     | 29       |
|    ent_coef        | 0.00689  |
|    ent_coef_loss   | 1.22     |
|    learning_rate   | 0.0003   |
|    n_updates       | 24139    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 244      |
|    fps             | 22       |
|    time_elapsed    | 1076     |
|    total timesteps | 24644    |
| train/             |          |
|    actor_loss      | 43.8     |
|    critic_loss     | 28.9     |
|    ent_coef 

...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 840      |
|    fps             | 23       |
|    time_elapsed    | 3652     |
|    total timesteps | 84840    |
| train/             |          |
|    actor_loss      | 50.2     |
|    critic_loss     | 38.5     |
|    ent_coef        | 0.00897  |
|    ent_coef_loss   | 1.4      |
|    learning_rate   | 0.0003   |
|    n_updates       | 84739    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 844      |
|    fps             | 23       |
|    time_elapsed    | 3670     |
|    total timesteps | 85244    |
| train/             |          |
|    actor_loss      | 50.6     |
|    critic_loss     | 19.4     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 900      |
|    fps             | 23       |
|    time_elapsed    | 3923     |
|    total timesteps | 90900    |
| train/             |          |
|    actor_loss      | 50       |
|    critic_loss     | 19.9     |
|    ent_coef        | 0.00513  |
|    ent_coef_loss   | -1.88    |
|    learning_rate   | 0.0003   |
|    n_updates       | 90799    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 904      |
|    fps             | 23       |
|    time_elapsed    | 3941     |
|    total timesteps | 91304    |
| train/             |          |
|    actor_loss      | 50.1     |
|    critic_loss     | 9.91     |
|    ent_coef 

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 960      |
|    fps             | 23       |
|    time_elapsed    | 4196     |
|    total timesteps | 96960    |
| train/             |          |
|    actor_loss      | 51       |
|    critic_loss     | 29.2     |
|    ent_coef        | 0.00438  |
|    ent_coef_loss   | -0.846   |
|    learning_rate   | 0.0003   |
|    n_updates       | 96859    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 964      |
|    fps             | 23       |
|    time_elapsed    | 4214     |
|    total timesteps | 97364    |
| train/             |          |
|    actor_loss      | 50       |
|    critic_loss     | 10.2     |
|    ent_coef 

position [ 0.874066   -0.76       -0.67567568 -0.52941176  0.93877551 -0.98089959
 -0.63076923 -0.63076923 -0.2         0.        ], action [0.874066], effect False, done False, global_reward -24
action [0.96536136]
position [ 0.96536136 -0.76       -0.67567568 -0.52941176  0.93877551 -0.98089959
 -0.63076923 -0.63076923 -0.2         0.        ], action [0.96536136], effect False, done False, global_reward -25
action [0.9551518]
position [ 0.9551518  -0.76       -0.67567568 -0.52941176  0.93877551 -0.98089959
 -0.63076923 -0.63076923 -0.2         0.        ], action [0.9551518], effect False, done False, global_reward -26
action [0.98472524]
position [ 0.98472524 -0.76       -0.67567568 -0.52941176  0.93877551 -0.98089959
 -0.63076923 -0.63076923 -0.2         0.        ], action [0.98472524], effect False, done False, global_reward -27
action [0.29568815]
position [ 0.29568815 -0.76       -0.67567568 -0.52941176  0.93877551 -0.98089959
 -0.63076923 -0.63076923 -0.2         0.        ],

position [ 0.9500246  -0.76       -0.67567568 -0.52941176  0.93877551 -0.98089959
 -0.63076923 -0.63076923 -0.2         0.        ], action [0.9500246], effect False, done False, global_reward -62
action [0.8876209]
position [ 0.88762093 -0.76       -0.67567568 -0.52941176  0.93877551 -0.98089959
 -0.63076923 -0.63076923 -0.2         0.        ], action [0.8876209], effect False, done False, global_reward -63
action [0.9898963]
position [ 0.9898963  -0.76       -0.67567568 -0.52941176  0.93877551 -0.98089959
 -0.63076923 -0.63076923 -0.2         0.        ], action [0.9898963], effect False, done False, global_reward -64
action [0.9649997]
position [ 0.96499968 -0.76       -0.67567568 -0.52941176  0.93877551 -0.98089959
 -0.63076923 -0.63076923 -0.2         0.        ], action [0.9649997], effect False, done False, global_reward -65
action [0.99591553]
position [ 0.99591553 -0.76       -0.67567568 -0.52941176  0.93877551 -0.98089959
 -0.63076923 -0.63076923 -0.2         0.        ], ac

# TD3 training

In [ ]:
from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

janus_env = Janus()
# check_env(janus_env)

n_actions = janus_env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))


model_janus_td3 = TD3("MlpPolicy", janus_env, action_noise=action_noise, verbose=2,tensorboard_log="./tensorboard/")
model_janus_td3.learn(total_timesteps=10000, log_interval=4, tb_log_name="janus partial td3")

janus_env.reset()
for i in range(100):
    action, _ = model_janus_td3.predict(janus_env.current_position)
    print(f'action {action}')
    obs, rewards, done, info = janus_env.step(action)
    janus_env.render()
    if done: break
janus_env.close()

features shape: (782, 228), 
targets shape: (782, 2)

Length of train is 703, test is 79
R squared: 0.6550
Output steps:  [0.064, 0.057]
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./tensorboard/janus partial td3_12
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 24.3     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 20       |
|    time_elapsed    | 0        |
|    total timesteps | 4        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 11.7     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 22       |
|    time_elapsed    | 0        |
|    total timesteps | 8        |
---------------------------------
---------------------------------
| rollout

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.0637   |
| time/              |          |
|    episodes        | 100      |
|    fps             | 22       |
|    time_elapsed    | 4        |
|    total timesteps | 100      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -0.946   |
| time/              |          |
|    episodes        | 104      |
|    fps             | 22       |
|    time_elapsed    | 4        |
|    total timesteps | 104      |
| train/             |          |
|    actor_loss      | -0.322   |
|    critic_loss     | 599      |
|    learning_rate   | 0.001    |
|    n_updates       | 3        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -0.948   |
| time/       

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 42.5     |
| time/              |          |
|    episodes        | 168      |
|    fps             | 19       |
|    time_elapsed    | 8        |
|    total timesteps | 168      |
| train/             |          |
|    actor_loss      | -26.8    |
|    critic_loss     | 4.25e+03 |
|    learning_rate   | 0.001    |
|    n_updates       | 67       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 45.5     |
| time/              |          |
|    episodes        | 172      |
|    fps             | 19       |
|    time_elapsed    | 8        |
|    total timesteps | 172      |
| train/             |          |
|    actor_loss      | -30.6    |
|    critic_loss     | 3.86e+03 |
|    learning_rate   | 0.001    |
|    n_updates       | 71       |
--------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 75.8     |
| time/              |          |
|    episodes        | 236      |
|    fps             | 19       |
|    time_elapsed    | 12       |
|    total timesteps | 236      |
| train/             |          |
|    actor_loss      | -71.5    |
|    critic_loss     | 2.52e+03 |
|    learning_rate   | 0.001    |
|    n_updates       | 135      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 74.8     |
| time/              |          |
|    episodes        | 240      |
|    fps             | 19       |
|    time_elapsed    | 12       |
|    total timesteps | 240      |
| train/             |          |
|    actor_loss      | -65.9    |
|    critic_loss     | 2.91e+03 |
|    learning_rate   | 0.001    |
|    n_updates       | 139      |
--------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 80.8     |
| time/              |          |
|    episodes        | 304      |
|    fps             | 18       |
|    time_elapsed    | 16       |
|    total timesteps | 304      |
| train/             |          |
|    actor_loss      | -91      |
|    critic_loss     | 2.16e+03 |
|    learning_rate   | 0.001    |
|    n_updates       | 203      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 80.8     |
| time/              |          |
|    episodes        | 308      |
|    fps             | 18       |
|    time_elapsed    | 16       |
|    total timesteps | 308      |
| train/             |          |
|    actor_loss      | -68.5    |
|    critic_loss     | 2.28e+03 |
|    learning_rate   | 0.001    |
|    n_updates       | 207      |
--------------

...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 70.7     |
| time/              |          |
|    episodes        | 4588     |
|    fps             | 16       |
|    time_elapsed    | 271      |
|    total timesteps | 4588     |
| train/             |          |
|    actor_loss      | -106     |
|    critic_loss     | 618      |
|    learning_rate   | 0.001    |
|    n_updates       | 4487     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 70.7     |
| time/              |          |
|    episodes        | 4592     |
|    fps             | 16       |
|    time_elapsed    | 271      |
|    total timesteps | 4592     |
| train/             |          |
|    actor_loss      | -104     |
|    critic_loss     | 598      |
|    learning_rate   | 0.001    |
|    n_updates       | 4491     |
--------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 71.7     |
| time/              |          |
|    episodes        | 4656     |
|    fps             | 16       |
|    time_elapsed    | 275      |
|    total timesteps | 4656     |
| train/             |          |
|    actor_loss      | -101     |
|    critic_loss     | 390      |
|    learning_rate   | 0.001    |
|    n_updates       | 4555     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 72.7     |
| time/              |          |
|    episodes        | 4660     |
|    fps             | 16       |
|    time_elapsed    | 276      |
|    total timesteps | 4660     |
| train/             |          |
|    actor_loss      | -106     |
|    critic_loss     | 703      |
|    learning_rate   | 0.001    |
|    n_updates       | 4559     |
--------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 61.6     |
| time/              |          |
|    episodes        | 4724     |
|    fps             | 16       |
|    time_elapsed    | 279      |
|    total timesteps | 4724     |
| train/             |          |
|    actor_loss      | -98.7    |
|    critic_loss     | 539      |
|    learning_rate   | 0.001    |
|    n_updates       | 4623     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 61.6     |
| time/              |          |
|    episodes        | 4728     |
|    fps             | 16       |
|    time_elapsed    | 280      |
|    total timesteps | 4728     |
| train/             |          |
|    actor_loss      | -95      |
|    critic_loss     | 739      |
|    learning_rate   | 0.001    |
|    n_updates       | 4627     |
--------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 67.7     |
| time/              |          |
|    episodes        | 4792     |
|    fps             | 16       |
|    time_elapsed    | 284      |
|    total timesteps | 4792     |
| train/             |          |
|    actor_loss      | -108     |
|    critic_loss     | 553      |
|    learning_rate   | 0.001    |
|    n_updates       | 4691     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 66.7     |
| time/              |          |
|    episodes        | 4796     |
|    fps             | 16       |
|    time_elapsed    | 284      |
|    total timesteps | 4796     |
| train/             |          |
|    actor_loss      | -102     |
|    critic_loss     | 202      |
|    learning_rate   | 0.001    |
|    n_updates       | 4695     |
--------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 68.7     |
| time/              |          |
|    episodes        | 4860     |
|    fps             | 16       |
|    time_elapsed    | 288      |
|    total timesteps | 4860     |
| train/             |          |
|    actor_loss      | -104     |
|    critic_loss     | 616      |
|    learning_rate   | 0.001    |
|    n_updates       | 4759     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 66.7     |
| time/              |          |
|    episodes        | 4864     |
|    fps             | 16       |
|    time_elapsed    | 289      |
|    total timesteps | 4864     |
| train/             |          |
|    actor_loss      | -101     |
|    critic_loss     | 368      |
|    learning_rate   | 0.001    |
|    n_updates       | 4763     |
--------------